In [ ]:
!pip install -r requirements.txt --user


# Inicialização

In [ ]:
from bbmagic import Spark, Hdfs, Kinit
import os
import getpass


In [ ]:
!kdestroy


# Parâmetros sessão Spark

In [ ]:
#CONSTANTES PARA DEFINIR A QUAL CLUSTER SE CONECTAR E O NOME DA SESSAO NO CONTEXTO DO NOTEBOOK E QUAL LINGUAGEM A SER UTILIZADA
HDP_VERSION = 3  # type: int
SESSION_NAME = "carga_sbx_d4q0b8f"  # type: str
LANGUAGE = "python"

#CONSTANTES QUE DEFINEM OS VALORES PARA DIMENSIONAMENTO DA SESSAO SPARK NO YARN
DRIVER_MEMORY = "2g"  # type: str
DRIVER_CORES = 2  # type: int
EXECUTOR_CORES = 3  # type: int
EXECUTOR_MEMORY = "6g"  # type: str
NUM_EXECUTORS = 1  # type: int 

#CONSTANTE QUE SERVE PARA CONFIGURAR PARAMETROSA DO SPARK CONFIG
SPARK_CONF = {
  "spark.driver.memoryOverhead": "1G",
  "spark.executor.memoryOverhead": "1G",
  "spark.sql.broadcastTimeout" : "6000000"
}  # type: Dict[str, str]

#CONSTANTE QUE SERVE PARA DEFINIR PARAMETROS A SEREM UTILIZADOS CONFORME AMBIENTE (MODELAGEM OU SCORAGE/PRODUCAO) NO QUAL O JOB SERA EXECUTADO 
AMBIENTE = os.environ["AMBIENTE"] 
if AMBIENTE == "MODELAGEM":
  ENV_VAR = {
    "DOMINIO":"sbx_d4q0b8f",
    "KEYTAB": input("Matrícula SISBB:"),
}
else:
  ENV_VAR = {
    "DOMINIO":"hive_d4q",
    "KEYTAB": os.environ['KEYTAB']
}


# Cria sessão Spark

In [ ]:
spark = Spark(
  session_name= SESSION_NAME, 
  username=  ENV_VAR["KEYTAB"], 
  hdp = HDP_VERSION, 
  language = LANGUAGE, 
  executor_memory=EXECUTOR_MEMORY,
  spark_conf=SPARK_CONF,
  auth = 'Kerberos',
  env = ENV_VAR
)


# Importacoes

In [ ]:
%%spark

import os



# Tabelas

##  *** opr_cred_sum ***

### Sql

In [ ]:
%%spark
sql1 = '''
SELECT --DISTINCT
1 as cd_ogn,
DT_REF, --DT_MVTC = DT_REF
CD_ABTR_LIM_NOVO, --CD_RAS_2022,
sum(VL_SDO_CTB) as VL_SDO_CTB,
sum(VL_PVS_CLCD_CTB) as VL_PVS_CLCD_CTB,
sum(VL_EXPO_ABGT) as VL_EXPO_ABGT, --VL_EXPO_ALT = VL_EXPO_ABGT
sum(VL_EXPO_PDRD_RSCO) as VL_EXPO_PDRD_RSCO, --VL_EXPO_PDRD_RSCO_ALT = VL_EXPO_PDRD_RSCO
sum(VL_PCL_PDRD_RSCO) as VL_PCL_PDRD_RSCO --VL_PCL_PDRD_RSCO_ALT = VL_PCL_PDRD_RSCO
FROM hive_d4q.cptl_rglo_msl
WHERE CD_IDFR_RBC_VLDO = 0 --IDFR_RBC_CTB_VLDO = CD_IDFR_RBC_VLDO
--AND dt_ref = '2021-12-31'
AND CD_QLF_TIP_OPR = 1
GROUP BY DT_REF, CD_ABTR_LIM_NOVO --, CD_RAS_2022
''' 



In [ ]:
%%spark
print(sql1)


### Dataframe

In [ ]:
%%time
%%spark

df1 = spark.sql(sql1).coalesce(20)
#dfData1 = df1.groupBy('aa_ref','mm_ref').count()
#dfData1.show()


### Nome da tabela

In [ ]:
%%spark

tabela1 = os.environ["DOMINIO"] + ".opr_cred_sum"


### Comando de exclusão das partições

In [ ]:
%%time
%%spark

#dfData1.cache()
#mes = int(dfData1.first()['mm_ref'])
#ano = int(dfData1.first()['aa_ref'])
#sqlParticao = "ALTER TABLE " + tabela1 + " DROP IF EXISTS PARTITION(AA_REF= " + str(ano) + ", MM_REF = " + str(mes) + ")"
#sqlParticao10 = "ALTER TABLE " + tabela1 + " DROP IF EXISTS PARTITION(AA_REF= " + str(ano-10) + ", MM_REF = " + str(mes) + ")"
#print(sqlParticao)
#print(sqlParticao10)


### Execução de exclusão das partições

In [ ]:
%%time
%%spark

#spark.sql(sqlParticao)
#spark.sql(sqlParticao10)
#spark.catalog.refreshTable(tabela1)


### Estrutura de fonte e destino

In [ ]:
%%spark

#dfDestino = spark.sql("select * from " + tabela1 + " limit 1")
#df1.printSchema()
#dfDestino.printSchema()


### Ordenação de campos. Totais antes e depois.

In [ ]:
%%spark

#dfOrdenada = df1.select(dfDestino.columns)

#dfOrdenada.groupBy('aa_ref','mm_ref').count().show()
#df1.groupBy('aa_ref','mm_ref').count().show()


### Inserindo dados na tabela

In [ ]:
%%time
%%spark

#dfOrdenada.write.insertInto(tabela1)
df1.write.mode("overwrite").saveAsTable(tabela1)


### Total da tabela

In [ ]:
%%spark

#print(tabela1)
#dfResult = spark.sql("select * from " + tabela1)
#dfResult.groupBy('aa_ref','mm_ref').count().show()


## Finaliza sessão Spark

In [ ]:
%spark cleanup
